In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    criterion: str
    max_features: str
    max_depth: int
    min_samples_leaf: int
    min_samples_split: int
    target_column: str

In [4]:
from src.mlReg.constants import *
from src.mlReg.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = Config_File_Path,
            params_filepath = Params_File_Path,
            schema_filepath = Schema_File_Path
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        dt_params = self.params.RandomForestRegressor
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            n_estimators = dt_params.n_estimators,
            criterion= dt_params.criterion,
            max_features= dt_params.max_features,
            max_depth= dt_params.max_depth,
            min_samples_leaf= dt_params.min_samples_leaf,
            min_samples_split= dt_params.min_samples_split,
            target_column= schema.name
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os
import joblib

from sklearn.ensemble import RandomForestRegressor
from src.mlReg.utils.logger import logger

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def training(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        dt = RandomForestRegressor(n_estimators=self.config.n_estimators, criterion= self.config.criterion, max_features= self.config.max_features,
                                   max_depth= self.config.max_depth, min_samples_leaf= self.config.min_samples_leaf, 
                                   min_samples_split= self.config.min_samples_split)
        dt.fit(X_train, y_train)

        joblib.dump(dt, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.training()
except Exception as e:
    raise e

[ [2024-03-11 16:13:28,323] : INFO : common : yaml file: config\config.yaml loaded successfully ]
[ [2024-03-11 16:13:28,423] : INFO : common : yaml file: params.yaml loaded successfully ]
[ [2024-03-11 16:13:28,510] : INFO : common : yaml file: schema.yaml loaded successfully ]
[ [2024-03-11 16:13:28,523] : INFO : common : created directory at: artifacts ]
[ [2024-03-11 16:13:28,543] : INFO : common : created directory at: artifacts/model_trainer ]
